In [35]:
!/Users/till/Downloads/google-cloud-sdk/bin/gcloud auth application-default login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=KcDlfGF8rRr3VBfitHuJNPOqh8GDjW&access_type=offline&code_challenge=tfarO2rM4qcHJNJLxGWCTuA0OnmNFzfC1ZIFpwp2TW0&code_challenge_method=S256


Credentials saved to file: [/Users/till/.config/gcloud/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "hackathum24mun-28" was added to ADC which can be used by Google client libraries for billing and quota. Note that some services may still bill the project owning the resource.


In [ ]:
!/Users/till/Downloads/google-cloud-sdk/bin/gcloud auth login

In [42]:
import base64
import vertexai
from vertexai.generative_models import GenerativeModel, SafetySetting, Part, Content

vertexai.init(project="hackathum24mun-28", location="europe-west3")
model = GenerativeModel(
    "gemini-1.5-flash-002",
)

generation_config = {
    "max_output_tokens": 8192,
    "temperature": 0.3,
    "top_p": 0.95,
}

safety_settings = [
    SafetySetting(
        category=SafetySetting.HarmCategory.HARM_CATEGORY_HATE_SPEECH,
        threshold=SafetySetting.HarmBlockThreshold.OFF
    ),
    SafetySetting(
        category=SafetySetting.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
        threshold=SafetySetting.HarmBlockThreshold.OFF
    ),
    SafetySetting(
        category=SafetySetting.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT,
        threshold=SafetySetting.HarmBlockThreshold.OFF
    ),
    SafetySetting(
        category=SafetySetting.HarmCategory.HARM_CATEGORY_HARASSMENT,
        threshold=SafetySetting.HarmBlockThreshold.OFF
    ),
]

In [47]:
response = model.generate_content(contents=[
    Content(role="user", parts=[Part.from_text("Say yes.")])
])

In [48]:
print(response.text)


Yes.



In [52]:
from langchain.agents import AgentType, Tool, initialize_agent
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate

model = GenerativeModel(
    "gemini-1.5-flash-002",
)

# Create memory for the conversation
conversation_memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Define the initial conversation prompt
life_planning_prompt = PromptTemplate(
    input_variables=["chat_history"],
    template="""You are a professional life planning advisor helping people make informed decisions about their future housing choices. Your role is to guide them through a comprehensive discussion about their life plans for the next 10 years, helping them uncover their true needs and preferences beyond their initial assumptions.

Start with these two questions:
1. What are your current living situation and immediate housing needs?
2. Where do you see yourself professionally and personally in the next 5-10 years?

After these initial questions, engage in a natural conversation that explores various life aspects relevant to housing choices, such as:
- Career growth and workplace location
- Family planning (marriage, children, pets)
- Lifestyle preferences (urban/suburban, social life, cultural activities)
- Daily necessities (transportation, shopping, healthcare)
- Educational needs (schools, universities)
- Work-life balance
- Budget and financial planning

Previous conversation:
{chat_history}

Respond professionally and empathetically. Help users discover what they truly need, not just what they think they want."""
)

# Create a preference extraction agent
preference_extractor_prompt = PromptTemplate(
    input_variables=["conversation"],
    template="""Based on the following conversation, extract and summarize the key preferences and requirements for housing:

{conversation}

Please provide a concise summary of:
1. Must-have features
2. Location preferences
3. Lifestyle requirements
4. Budget constraints
5. Long-term considerations

Format the response as a clear, structured summary."""
)

def get_user_input():
    return input("You: ")

def chat_with_user():
    chat_history = []
    
    # Initialize conversation with the life planning prompt
    while True:
        # Get the next prompt
        prompt = life_planning_prompt.format(chat_history="\n".join(chat_history))
        
        # Get AI response
        response = model.generate_content(
            contents=[Content(role="user", parts=[Part.from_text(prompt)])]
        ).text
        print("\nAdvisor:", response)
        
        # Get user input
        user_input = get_user_input()
        
        # Check if user wants to end conversation
        if user_input.lower() in ['exit', 'quit', 'done', 'next']:
            break
            
        # Update chat history
        chat_history.extend([f"User: {user_input}", f"Advisor: {response}"])
    
    return chat_history

def extract_preferences(conversation):
    prompt = preference_extractor_prompt.format(conversation="\n".join(conversation))
    summary = model.generate_content(
        contents=[Content(role="user", parts=[Part.from_text(prompt)])]
    ).text
    return summary

# Main execution flow
print("Welcome to your Life Planning Advisor!")
print("Let's discuss your future housing needs and life plans.")
print("(Type 'exit', 'quit', 'done', or 'next' when you're ready to move to the preference summary)")

# Start the conversation
conversation_history = chat_with_user()

# Extract and display preferences
print("\nBased on our conversation, here's a summary of your preferences and requirements:")
preferences_summary = extract_preferences(conversation_history)
print(preferences_summary)


Welcome to your Life Planning Advisor!
Let's discuss your future housing needs and life plans.
(Type 'exit', 'quit', 'done', or 'next' when you're ready to move to the preference summary)

Advisor: Okay, let's begin planning your future housing journey.  To start, I have two questions to help us frame things:

1. **What is your current living situation and what are your immediate housing needs?**  (For example:  Are you renting an apartment? Owning a house? Sharing a house? Do you need more space? Are there any immediate repairs or maintenance issues?)

2. **Where do you see yourself professionally and personally in the next 5-10 years?** (This could include career aspirations, relationship goals, family plans, etc.  Be as specific or broad as you feel comfortable.)


Once we've established a baseline, we can delve deeper into other factors.  I want to understand not just what you *think* you want in a home, but what will truly support your overall well-being and aspirations over the n